In [11]:
import numpy as np

def calculate_elo_probability(elo_a, elo_b, home_advantage=0):

    elo_a += home_advantage
    prob_a = 1 / (1 + 10 ** ((elo_b - elo_a) / 400))
    return prob_a

def calculate_recent_performance(team_results):
    
    weights = np.array([5, 4, 3, 2, 1])
    performance_score = np.dot(team_results, weights) / sum(weights)
    return performance_score

def calculate_head_to_head_performance(h2h_result):
    
    return h2h_result

def calculate_statistical_adjustments(stats_a, stats_b):
    
    adjustment = 0
    factors = ['goals_scored', 'goals_conceded', 'possession', 'shots_on_target']
    weights = {'goals_scored': 0.4, 'goals_conceded': -0.3, 'possession': 0.2, 'shots_on_target': 0.1}

    for factor in factors:
        adjustment += weights[factor] * (stats_a[factor] - stats_b[factor])

    return adjustment / len(factors)

def calculate_tilt(team_results, stats):
    performance = calculate_recent_performance(team_results)
    
    # Możemy na przykład użyć średniej wartości statystyk do obliczenia Tilt
    average_goals_scored = stats['goals_scored']
    average_goals_conceded = stats['goals_conceded']
    
    # Przykładowe wagi do obliczenia Tilt
    performance_weight = 0.5
    goals_scored_weight = 0.3
    goals_conceded_weight = 0.2
    
    # Obliczenie Tilt jako ważona suma statystyk i wyników
    tilt = (performance_weight * performance +
            goals_scored_weight * average_goals_scored -
            goals_conceded_weight * average_goals_conceded)
    
    return tilt

def calculate_match_probability(elo_a, elo_b, home, team_a_results, team_b_results, h2h_result, stats_a, stats_b, tilt_a, tilt_b, margin):
    
    home_advantage = 100 if home else 0
    base_probability = calculate_elo_probability(elo_a, elo_b, home_advantage)
    
    perf_a = calculate_recent_performance(team_a_results)
    perf_b = calculate_recent_performance(team_b_results)
    perf_adjustment = (perf_a - perf_b) / 2
    
    h2h_adjustment = calculate_head_to_head_performance(h2h_result)
    
    stats_adjustment = calculate_statistical_adjustments(stats_a, stats_b)
    
    expected_goals = 2.5
    tilt_adjustment = 0.98 * tilt_a + 0.02 * margin / (tilt_b * expected_goals)
    
    total_adjustment = perf_adjustment + h2h_adjustment * 0.1 + stats_adjustment * 0.1 + tilt_adjustment * 0.1
    
    final_probability = base_probability + total_adjustment * 0.1
    
    final_probability = max(0, min(1, final_probability))
    
    return final_probability

def monte_carlo_simulation(num_simulations, elo_a, elo_b, home, team_a_results, team_b_results, h2h_result, stats_a, stats_b, tilt_a, tilt_b, margin):
    wins_a = 0
    wins_b = 0
    draws = 0
    for _ in range(num_simulations):
        prob_a_wins = calculate_match_probability(elo_a, elo_b, home, team_a_results, team_b_results, h2h_result, stats_a, stats_b, tilt_a, tilt_b, margin)
        result = np.random.rand()
        if result < prob_a_wins:
            wins_a += 1
        elif result < prob_a_wins + (1 - prob_a_wins) / 2:
            draws += 1
        else:
            wins_b += 1
    return wins_a / num_simulations, draws / num_simulations, wins_b / num_simulations

def main():
    # Pobieranie danych wejściowych od użytkownika
    elo_a = int(input("Podaj ranking ELO drużyny A: "))
    elo_b = int(input("Podaj ranking ELO drużyny B: "))
    home = input("Czy drużyna A gra u siebie? (tak/nie): ").strip().lower() == 'tak'
    team_a_results = list(map(int, input("Podaj wyniki z ostatnich 5 meczów drużyny A (oddzielone spacjami): ").split()))
    team_b_results = list(map(int, input("Podaj wyniki z ostatnich 5 meczów drużyny B (oddzielone spacjami): ").split()))
    h2h_result = int(input("Podaj wynik ostatniego bezpośredniego meczu (1 - wygrana drużyny A, 0 - remis, -1 - wygrana drużyny B): "))
    stats_a = {
        'goals_scored': float(input("Podaj średnią liczbę strzelonych bramek drużyny A: ")),
        'goals_conceded': float(input("Podaj średnią liczbę straconych bramek drużyny A: ")),
        'possession': float(input("Podaj średni procent posiadania piłki przez drużynę A: ")),
        'shots_on_target': float(input("Podaj średnią liczbę strzałów celnych drużyny A: "))
    }
    stats_b = {
        'goals_scored': float(input("Podaj średnią liczbę strzelonych bramek drużyny B: ")),
        'goals_conceded': float(input("Podaj średnią liczbę straconych bramek drużyny B: ")),
        'possession': float(input("Podaj średni procent posiadania piłki przez drużynę B: ")),
        'shots_on_target': float(input("Podaj średnią liczbę strzałów celnych drużyny B: "))
    }
    
    # Obliczanie wartości Tilt dla obu drużyn
    tilt_a = calculate_tilt(team_a_results, stats_a)
    tilt_b = calculate_tilt(team_b_results, stats_b)
    
    margin = float(input("Podaj różnicę bramek: "))
    num_simulations = int(input("Podaj liczbę symulacji: "))

    # Przeprowadzenie symulacji
    prob_a_wins, prob_draw, prob_b_wins = monte_carlo_simulation(num_simulations, elo_a, elo_b, home, team_a_results, team_b_results, h2h_result, stats_a, stats_b, tilt_a, tilt_b, margin)

    print(f"Prawdopodobieństwo wygranej drużyny A: {prob_a_wins:.2f}")
    print(f"Prawdopodobieństwo remisu: {prob_draw:.2f}")
    print(f"Prawdopodobieństwo wygranej drużyny B: {prob_b_wins:.2f}")

if __name__ == "__main__":
    main()


Prawdopodobieństwo wygranej drużyny A: 0.63
Prawdopodobieństwo remisu: 0.19
Prawdopodobieństwo wygranej drużyny B: 0.18
